In [28]:
import pymysql

db = pymysql.connect(host= 'localhost', port = 3306, user = 'root', passwd='2031mysq!', db='bestproducts', charset='utf8')
cursor = db.cursor()

sql = '''
CREATE TABLE items (
    item_code VARCHAR(20) NOT NULL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    ori_price INT NOT NULL,
    dis_price INT NOT NULL,
    discount_percent INT NOT NULL,
    provider VARCHAR(100)
);
'''
cursor.execute(sql)

sql = '''
CREATE TABLE ranking (
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
    main_category VARCHAR(50) NOT NULL,
    sub_category VARCHAR(50) NOT NULL,
    item_ranking TINYINT UNSIGNED NOT NULL,
    item_code VARCHAR(20) NOT NULL,
    FOREIGN KEY (item_code) REFERENCES items(item_code)
);
'''
cursor.execute(sql)

db.commit()
db.close()

OperationalError: (1050, "Table 'items' already exists")

In [22]:
def get_category(category_link, category_name):
    print (category_link, category_name)
    res = requests.get(category_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    
    get_items(soup, category_name, "ALL")
    
    sub_categories = soup.select('div.navi.group ul li > a')
    for sub_category in sub_categories:
        res = requests.get('https://www.gmarket.co.kr' + sub_category['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        get_items(soup, category_name, sub_category.get_text())


In [23]:
def get_items(html, category_name, sub_category_name):
    items_result_list = list()
    bestitems = html.select_one('div.best-list')
    products = bestitems.select('ul > li')
    
    for index, item in enumerate(products):
        data_dict = dict()
        ranking = index + 1
        title = item.select_one('a.itemname')
        ori_price = item.select_one('div.o-price')
        dis_price = item.select_one('div.s-price strong span')
        discount_percent = item.select_one('div.s-price em')
    
        # ori_price.get_text() == ''는 텍스트는 있지만 데이터는 없는 경우
        if ori_price == None or ori_price.get_text() == '':
            ori_price = dis_price
        if dis_price == None:
            ori_price, dis_price = 0, 0
        # ex. 15,900원 -> 15900 변경하는 과정
        else:
            ori_price = ori_price.get_text().replace(',', '').replace('원', '')
            dis_price = dis_price.get_text().replace(',', '').replace('원', '')

        if discount_percent == None or discount_percent.get_text() == '':
            discount_percent = 0
         # ex. 70% -> 70 변경하는 과정
        else:
            discount_percent = discount_percent.get_text().replace('%', '')
        
            # 판매자 정보 가져오기
            product_link = item.select_one('div.thumb > a')
        # '='를 기준으로 앞에가 0, 뒤에가 1. 1에 있는 정보를 가져오기
        item_code = product_link.attrs['href'].split('=')[1].replace('&ver','')
    
        res = requests.get(product_link.attrs['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        provider = soup.select_one('div.item-topinfo_headline > p > span > a')
        if provider == None:
            provider = ''
        else:
            provider = provider.get_text()

        data_dict['category_name'] = category_name
        data_dict['sub_category_name'] = sub_category_name
        data_dict['ranking'] = ranking
        data_dict['title'] = title.get_text()
        data_dict['ori_price'] = ori_price
        data_dict['dis_price'] = dis_price
        data_dict['discount_percent'] = discount_percent
        data_dict['item_code'] = item_code
        data_dict['provider'] = provider

        save_data(data_dict)
        # print (category_name, sub_category_name, ranking, item_code, provider, title.get_text(), ori_price,  discount_percent)

In [30]:
import requests
from bs4 import BeautifulSoup
import pymysql

db = pymysql.connect(host= 'localhost', port = 3306, user = 'root', passwd='2031mysq!', db='bestproducts', charset='utf8')
cursor = db.cursor()

res = requests.get('https://www.gmarket.co.kr/n/best')
soup = BeautifulSoup(res.content, 'html.parser')

categories = soup.select('div.gbest-cate ul.by-group li a')
for category in categories:
    get_category('https://www.gmarket.co.kr' + category['href'], category.get_text())
    
# 실행속도가 매우 느리다는 단점
# pycharm 을 이용하면 빠르게 사용할 수 있음 (자동저장도 제공)
# 저장된 디렉터리 파일을 바로 이용하기도 함

https://www.gmarket.co.kr/n/best ALL
{'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[비비고]비비고 갈비탕 400g 5봉', 'ori_price': '정가26500', 'dis_price': '21200', 'discount_percent': '20', 'item_code': '1553421610', 'provider': '스마일배송'}
INSERT INTO items VALUES('1553421610',
        '[비비고]비비고 갈비탕 400g 5봉',
        정가26500,
        21200,
        20,
        '스마일배송')


OperationalError: (1054, "Unknown column '정가26500' in 'field list'")

### Insert SQL 만들기

#### Table 구조
CREATE TABLE ranking (  
    num INT AUTO_INCREMENT NOT NULL PRIMARY KEY,  
    main_category VARCHAR(50) NOT NULL,  
    sub_category VARCHAR(50) NOT NULL,  
    item_ranking TINYINT UNSIGNED NOT NULL,  
    item_code VARCHAR(20) NOT NULL,  
    FOREIGN KEY (item_code) REFERENCES items(item_code)  
);

In [15]:
item_info = {'category_name': 'ALL', 'sub_category_name': 'ALL', 'ranking': 1, 'title': '[파격조건]고메 함박스테이크 152gX10개', 'ori_price': '정가20000', 'dis_price': '19000', 'discount_percent': '5', 'item_code': '1718892809', 'provider': 'CJ온스타일LIVE'}

In [ ]:
sql = """INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('
"""+item_info['category_name']+"""',
'"""+item_info['sub_category_name']+"""',
'"""+str(item_info['ranking'])+"""',
'"""+item_info['item_code']+"""')"""
# 해당 sql구문을 그대로 넣으면 무결성의 원리에 따라 에러가 발생하게 됨
# 따라서 items를 먼저 생성해야 함

In [ ]:
sql.replace('\n', '')

### 목적
- 실제로 데이터베이스를 사용하는 방법을 현업스타일로 설명을 드리는 것
- 현실 세계의 데이터를 어떻게 테이블로 정의할 것인가
- 어떻게 데이터를 넣을 것인가? (프로그래밍으로 넣는 것이 일반적)
- 저장된 데이터를 분석 (SQL만 쓰는 경우 + 프로그래밍에서 데이터를 분석해서, 풀스택으로 서비스화함)

#### 기존 SQL 강의: 단순히 세세한 SQL 문법을 다 소개받고, 끝

### Table 구조
CREATE TABLE items (  
item_code VARCHAR(20) NOT NULL PRIMARY KEY,  
title VARCHAR(200) NOT NULL,  
ori_price INT NOT NULL,  
dis_price INT NOT NULL,  
discount_percent INT NOT NULL,  
provider VARCHAR(100)  
);

In [ ]:
sql = """INSERT INTO items VALUES('
"""+item_info['item_code']+"""
''"""+item_info['title']+"""',
"""+item_info['ori_price']+""",
"""+item_info['dis_price']+""",
"""+item_info['discount_percent']+""",
'"""+item_info['provider']+"""')"""
# 이후 ranking에 입력하는 구문이 실행 가능해짐
# 이런 식으로 테스트를 한 후 입력 구문을 넣어야 함

In [ ]:
sql.replace('\n', '')

#### DELETE TABLE DATE

In [ ]:
import pymysql

db = pymysql.connect(host= 'localhost', port = 3306, user = 'root', passwd='2031mysq!', db='bestproducts', charset='utf8')
cursor = db.cursor()

sql = 'DELETE FROM ranking'
cursor.execute(sql)
# 무결성이 깨지기 때문에 ranking 을 먼저 삭제해야 함

sql = 'DELETE FROM items'
cursor.execute(sql)

db.commit()
db.close()

In [27]:
def save_data(item_info):
    print(item_info)
    sql = """SELECT COUNT(*) FROM items WHERE   item_code='"""+item_info['item_code']+"""';"""  
    cursor.execute(sql)   
    result = cursor.fetchone()
    if result[0] == 0:
        sql = """INSERT INTO items VALUES('"""+item_info['item_code']+"""',
        '"""+item_info['title']+"""',
        """+str(item_info['ori_price'])+""",
        """+str(item_info['dis_price'])+""",
        """+str(item_info['discount_percent'])+""",
        '"""+item_info['provider']+"""')"""
        print(sql)
        cursor.execute(sql)
        
    sql = """INSERT INTO ranking(main_category, sub_category, item_ranking, item_code) VALUES('
    """+item_info['category_name']+"""',
    '"""+item_info['sub_category_name']+"""',
    '"""+str(item_info['ranking'])+"""',
    '"""+item_info['item_code']+"""')"""
    print(sql)
    cursor.execute(sql)

#### COUNT SQL
- COUNT: 검색 결과의 row 수를 가져올 수 있는 SQL 문법
- SQL 예제 : SELECT COUNT(*) FROM items
- 필드에 해당 값이 얼마나 들어있는지를 알려줌  
'''sql  
sql = """SELECT COUNT(*) FROM items WHERE   item_code='"""+item_info['item_code']+"""';"""  
cursor.execute(sql)   
result = cursor.fetchone()  
print (result[0]) # 어차피 하나 들어가 있으므로 해당 값을 지칭하려면 [0]을 해야함  
'''

In [16]:
sql = """SELECT COUNT(*) FROM items WHERE item_code= '"""+item_info['item_code']+"""';"""

In [17]:
sql

"SELECT COUNT(*) FROM items WHERE item_code= '1718892809';"